In [0]:
from tweepy import OAuthHandler

def tryAuth(consumerKey:str, 
            consumerSecret:str, 
            accessToken:str, 
            tokenSecret:str):
    try:
        auth = OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, tokenSecret)
        return auth
    except tweepy.TweepError as err:
        print("Failed Connection! \n", err)
        return None
    except Exception as e:
        print(e)
        return None  

def getCredentialsNauthenticate(tokenFile):  
    tokenFile.readline()
    for line in tokenFile:
        line = line.rstrip()
        key = line.split(',')
        consumerKey = key[0]
        consumerSecret = key[1]
        accessToken = key[2]
        tokenSecret = key[3]

    return tryAuth(consumerKey,
                   consumerSecret,
                   accessToken,
                   tokenSecret)

In [0]:
#single client twitter socket
import socket
import tweepy
import json

from tweepy.streaming import StreamListener
from tweepy import Stream
from typing import Dict, List, Tuple

class ListenerTS(StreamListener):
  def __init__(self, clientSocket):
    self.clientSocket = clientSocket

  def on_data(self, rawData):
    try:
      data = rawData.encode('utf8')
      self.clientSocket.send(data)
      return True
    except BaseException as e:
            # Error handling
            print("Ahh! Look what is wrong : %s" % str(e))
            return True

  def on_error(self, status):
      print(status)
      return True

class TwitterSocketStreamer(object):
  """SET UP STREAMER"""
  def __init__(self):
      self._auth = None
      self._serverSocket = None
      self._clientSocket = None

  def APIAuthenticate(self, tokenFile=None, 
                  consumerKey:str=None, 
                  consumerSecret:str=None, 
                  accessToken:str=None,
                  accessTokenSecret:str=None):
    if tokenFile is not None:
        self._auth = getCredentialsNauthenticate(tokenFile)

    elif consumerKey is not None and\
       consumerSecret is not None and\
       accessToken is not None and\
       accessTokenSecret is not None:
        self._auth = tryAuth(consumerKey,
                        consumerSecret,
                        accessToken,
                        tokenSecret) 
    else:
      raise Exception("Error: Neither credentials, nor token file were provided.")

  def setSocketConnection(self, host:str, port:int):
      if self._serverSocket is None:
          self._serverSocket = socket.socket()
          self._serverSocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
          self._serverSocket.bind((host, port))
          print("Listening on port: %s" % str(port))
          self._serverSocket.listen(5)
          self._clientSocket, addr = self._serverSocket.accept()
          print("Received request from: " + str(addr))
      else:
        print("Socket aldready opened. Reusing...")

  def closeConnection(self):
      self._clientSocket.close()
      self._serverSocket.close()

  def filterTweetsBy(self, keywords:List[str]=None,
                     locations:List[Tuple[float]]=None,
                     delimited=None,
                     follow:list=None,
                     filter_level=None,
                     languages:List[str]=None,
                     ):
      self.filters = {
          "track" : keywords,
          "locations": locations,
          "follow": follow,
          "languages": languages,
        "filter_level": filter_level
      }

  def streamTweets(self):
      while True:
          listener = ListenerTS(self._clientSocket) 
          stream = Stream(self._auth, listener)
          stream.filter(**self.filters)